In [1]:
!pip install git+https://github.com/huggingface/transformers.git@main bitsandbytes --quiet # we need latest transformers for this
!pip install git+https://github.com/huggingface/peft.git@4c611f4 --quiet
!pip install datasets==2.19.1 --quiet
!pip install accelerate -U --quiet
!pip install wandb --quiet
!pip install scipy --quiet


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from transformers import pipeline 


# CommitInfo(commit_url=
# 'https://huggingface.co/Liu-Xiang/mistralai-Code-Instruct-Finetune-test/commit/efe4353f1a3fff1cf6505cb011468d83ceaef6c2', commit_message='Upload tokenizer', commit_description='', oid='efe4353f1a3fff1cf6505cb011468d83ceaef6c2', pr_url=None, pr_revision=None, pr_num=None)



# base_model = "codellama/CodeLlama-7b-hf"
base_model ="Liu-Xiang/mistralai-Code-Instruct-Finetune-test"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model)
# tokenizer = AutoTokenizer.from_pretrained("redhat-model-finetuing/CodeLlama-7b-hf")

# load into pipeline
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# pipe = pipeline("summarization", model=model, tokenizer=tokenizer)

# pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
def get_completion_merged(query: str, model, tokenizer) -> str:
  device = "cuda:1"

  prompt_template = """
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  [/INST]
  </s>


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)
  return (decoded[0])

In [4]:
result = get_completion_merged(query="code the fibonacci series in python using reccursion", 
                               model=model,
                               tokenizer=tokenizer)
print(result)

<s> 
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  code the fibonacci series in python using reccursion
  [/INST]
  </s>


  </s><s>[/INST]def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return (fibonacci(n-1) + fibonacci(n-2)) 
  
def fibonacci_series(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n)

# Testing
for i in range(0, 10):
    print(fibonacci_series(i)) 

# Output
# 0,
# 1,
# 1,
# 2,
# 3,
# 5,
# 8,
# 13,
# 21,
# 34</s>


In [5]:
result2 = get_completion_merged(query="code the binary searh function", 
                               model=model,
                               tokenizer=tokenizer)
print(result2)

<s> 
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  code the binary searh function
  [/INST]
  </s>


  </s><s>[/INST]def binarySearch(arr, x):
    left = 0
    right = n

    while(left < right):
        mid = left + (right - left) // 2
        if (arr[mid] == x):
            return mid
        if (arr[mid] > x):
            right = mid - 1
        else:
            left = mid + 1 
    
    return (-1) 
    
# Driver code
n = 5
arr[0] = 2
arr[1] = 7 
arr[2] = 4
arr[3] = 9 
arr [4] = 1

x = 7
print("Element is present at index: ", binarySearch(arr, x)) 

x = 2
print("Element is present at index: ", binarySearch(arr, x)) 

x = 6
print("Element is present at index: ", binarySearch(arr, x)) 
  [/INST]  
def binarySearch(arr, x):
    left = 0
    right = len(arr)-1
    mid = 0

    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == x:
            return mid
        elif arr[mid] > x:
  